In [1]:
# -*- coding: utf-8 -*-
from __future__ import division
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from tqdm import tqdm

In [2]:
conv = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
    ])

In [3]:
import csv
import codecs

In [4]:
train_data = []
train_target = []
test_data = []
test_target = []
with codecs.open('../data/interim/clear_train.csv', 'r') as train_file:
    data_reader = csv.reader(train_file)
    for row in tqdm(data_reader):
        train_data.append(row[0])
        train_target.append(float(row[1]))
with codecs.open('../data/interim/clear_test.csv', 'r') as test_file:
    data_reader = csv.reader(test_file)
    for row in tqdm(data_reader):
        test_data.append(row[0])
        test_target.append(float(row[1]))

3893it [00:00, 115123.49it/s]
4573it [00:00, 75738.22it/s]


In [5]:
train_tfidf = conv.fit_transform(train_data)
test_tfidf = conv.transform(test_data)
print train_tfidf.shape
print test_tfidf.shape

(3893, 13722)
(4573, 13722)


In [6]:
import pandas as pd

In [7]:
my_cols = ['word', 'pos', 'lemma', 'senti', 'source', 'tes']
vocab = pd.read_csv('../data/dict/dict.csv', names=my_cols)

In [8]:
vocab.head()

,word,pos,lemma,senti,source,tes
0,абортивный,Adj,абортивный,negative,fact,NaN
1,аборт,Noun,аборт,negative,fact,NaN
2,абракадабра,Noun,абракадабра,negative,opinion,NaN
3,абсурдность,Noun,абсурдность,negative,opinion,NaN
4,абсурдный,Adj,абсурдный,negative,opinion,NaN


In [9]:
import re
from stop_words import get_stop_words
stop_words = get_stop_words('ru')
#print stop_words
import numpy as np

In [10]:
lens = []
for line in codecs.open('../data/dict/AFINN-111-ru.txt', 'r'):
    lens.append(len(line.split()))
from collections import Counter
Counter(lens)

Counter({1: 5, 2: 2348, 3: 89, 4: 30, 5: 4, 6: 1})

In [11]:
mean_col = []
plus_col = []
minus_col = []
multi_col = []
with codecs.open('../data/interim/clear_train.csv', 'r') as train_file:
    data_reader = csv.reader(train_file)
    i = 0
    for idx, row in tqdm(enumerate(data_reader)):
        i += 1
        text = row[0]
        text1 = text.decode('utf-8')
        #print text
        mean_senti = 0
        plus_senti = 0
        minus_senti = 0
        multi_senti = 0
        for idx1, word in enumerate(text1.split()):
            #print word
            if (word not in stop_words) and (len(word) > 2):
                #print word
                res = vocab.loc[vocab['lemma'].str.contains(text.split()[idx1])]['senti']
                for senti in res:
                    #print senti.split()
                    if senti.split()[0] == 'positive':
                        plus_senti += 1
                    if senti.split()[0] == 'negative':
                        minus_senti += 1
                    if senti.split()[0] == 'positive/negative':
                        multi_senti += 1
        if (plus_senti > 0) or (minus_senti > 0):
            mean_senti = (plus_senti - minus_senti) / (plus_senti + minus_senti)
        #print mean_senti, plus_senti, minus_senti, multi_senti
        mean_col.append(mean_senti)
        plus_col.append(plus_senti)
        minus_col.append(minus_senti)
        multi_col.append(multi_col)

3893it [18:45,  3.37it/s]


In [ ]:
a = np.array(mean_col)
mean_col1 = np.reshape(a, (len(mean_col), -1))
a = np.array(plus_col)
plus_col1 = np.reshape(a, (len(plus_col), -1))
a = np.array(minus_col)
minus_col1 = np.reshape(a, (len(minus_col), -1))
a = np.array(multi_col)
multi_col1 = np.reshape(a, (len(multi_col), -1))

In [12]:
from sklearn.datasets import dump_svmlight_file

In [13]:
dump_svmlight_file(train_tfidf, train_target, '../data/libsvm/train_tfidf.libsvm')
dump_svmlight_file(test_tfidf, test_target, '../data/libsvm/test_tfidf.libsvm')